In [2]:
# 기본
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import matplotlib
from matplotlib import font_manager, rc
import platform
from tqdm import tqdm
import sklearn
from sklearn import linear_model
import scipy.stats as stats
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor
from patsy import dmatrices
from sklearn.model_selection import train_test_split

# time
import datetime

# crawling
import requests
import lxml.html
import sqlite3
from pandas.io import sql
from bs4 import BeautifulSoup

# 한글 폰트 설정
if platform.system() == 'Windows':
# 윈도우인 경우
    font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
    rc('font', family=font_name)

In [3]:
aus_13 = pd.read_excel('D:\\SNUlab\\0. data\\thermal inversion\\Australia_Victoria_Air\\2013_All_sites_air_quality_hourly_avg_AIR-I-F-V-VH-O-S1-DB-M2-4-0.xlsx', 'Air Quality Data')
aus_14 = pd.read_excel('D:\\SNUlab\\0. data\\thermal inversion\\Australia_Victoria_Air\\2014_All_sites_air_quality_hourly_avg_AIR-I-F-V-VH-O-S1-DB-M2-4-0.xlsx', 'Air Quality Data')
aus_15 = pd.read_excel('D:\\SNUlab\\0. data\\thermal inversion\\Australia_Victoria_Air\\2015-All sites air quality hourly avg. AIR-I- F-V-VH-O-S1-DB-M2-4-0.xlsx', 'Data')
aus_16 = pd.read_excel('D:\\SNUlab\\0. data\\thermal inversion\\Australia_Victoria_Air\\2016-All sites air quality hourly avg. AIR-I- F-V-VH-O-S1-DB-M2-4-0.xlsx', 'Data')
aus_17 = pd.read_excel('D:\\SNUlab\\0. data\\thermal inversion\\Australia_Victoria_Air\\2017-All sites air quality hourly avg. AIR-I- F-V-VH-O-S1-DB-M2-4-0.xlsx', 'Data')

In [4]:
# 13, 14년 date 생성 함수

def create_date(df):
    year = df['sample_datetime'][7:11]
    month = df['sample_datetime'][0:3]
    day = df['sample_datetime'][4:6]

    if month == 'Jan': 
        month = '01'
    elif month == 'Feb': 
        month = '02'
    elif month == 'Mar': 
        month = '03'
    elif month == 'Apr': 
        month = '04'
    elif month == 'May': 
        month = '05'
    elif month == 'Jun': 
        month = '06'
    elif month == 'Jul': 
        month = '07'
    elif month == 'Aug': 
        month = '08'
    elif month == 'Sep': 
        month = '09'
    elif month == 'Oct': 
        month = '10'
    elif month == 'Nov': 
        month = '11'
    elif month == 'Dec': 
        month = '12'

    if day in [' 1', ' 2', ' 3', ' 4', ' 5', ' 6', ' 7', ' 8', ' 9']: 
        day = day[1:]
        date = year + '-' + month + '-' + '0' + day
    else: 
        date = year + '-' + month + '-' + day

    return date

In [5]:
aus_13['param_short_name'].unique()

array(['Visibility Reduction', 'PM10', 'CO', 'NO2', 'O3', 'SO2', nan],
      dtype=object)

In [6]:
aus_14['param_short_name'].unique()

array(['Visibility Reduction', 'CO', 'NO2', 'O3', 'PM10', 'SO2', 'PM2.5',
       nan], dtype=object)

In [7]:
aus_15['param_short_name'].unique()

array(['Visibility Reduction', 'PM2.5', 'CO', 'NO2', 'O3', 'PM10', nan,
       'SO2'], dtype=object)

In [8]:
# 년도별 param_short_name in air_pollutant, sample_datetime == '10:00 AM' 만 뽑아서 concat 하기
# aus_13, aus_14 는 'PV' -> 'value' 로 column명 change

air_pollutants = ['CO', 'NO2', 'O3', 'PM10', 'SO2', 'PM2.5']

aus_13_api = aus_13.loc[aus_13['param_short_name'].isin(air_pollutants) == True].reset_index(drop = True)
aus_13_api = aus_13_api.loc[aus_13_api['sample_datetime'].str[0:3] == 'Dec'].reset_index(drop = True) # 13년은 12월만 뽑기
aus_13_api = aus_13_api.loc[aus_13_api['sample_datetime'].str[12:] == '10:00AM'].reset_index(drop = True)
aus_13_api['date'] = aus_13_api.apply(create_date, axis = 1)
aus_13_api.rename(columns = {'PV' : 'value'}, inplace = True)

aus_14_api = aus_14.loc[aus_14['param_short_name'].isin(air_pollutants) == True].reset_index(drop = True)
aus_14_api = aus_14_api.loc[aus_14_api['sample_datetime'].str[12:] == '10:00AM'].reset_index(drop = True)
aus_14_api['date'] = aus_14_api.apply(create_date, axis = 1)
aus_14_api.rename(columns = {'PV' : 'value'}, inplace = True)

aus_15_api = aus_15.loc[aus_15['param_short_name'].isin(air_pollutants) == True].reset_index(drop = True)
aus_15_api = aus_15_api.loc[aus_15_api['sample_datetime'].dt.strftime("%H") == '10'].reset_index(drop = True)
aus_15_api['date'] = aus_15_api['sample_datetime'].dt.strftime('%Y-%m-%d')

aus_16_api = aus_16.loc[aus_16['param_short_name'].isin(air_pollutants) == True].reset_index(drop = True)
aus_16_api = aus_16_api.loc[aus_16_api['sample_datetime'].dt.strftime("%H") == '10'].reset_index(drop = True)
aus_16_api['date'] = aus_16_api['sample_datetime'].dt.strftime('%Y-%m-%d')

aus_17_api = aus_17.loc[aus_17['param_short_name'].isin(air_pollutants) == True].reset_index(drop = True)
aus_17_api = aus_17_api.loc[aus_17_api['sample_datetime'].dt.strftime("%H") == '10'].reset_index(drop = True)
aus_17_api['date'] = aus_17_api['sample_datetime'].dt.strftime('%Y-%m-%d')

aus_1317_api = pd.concat([aus_13_api, aus_14_api, aus_15_api, aus_16_api, aus_17_api], axis = 0)
aus_1317_api.reset_index(drop = True, inplace = True)

In [9]:
aus_1317_api

,sample_point_id,sp_name,latitude,longitude,sample_datetime,time_basis_id,param_id,param_name,value,param_std_unit_of_measure,param_short_name,date
0,10001,Alphington,-37.778316,145.030594,Dec 1 2013 10:00AM,1HR_AV,CO,Carbon Monoxide,0.1,ppm,CO,2013-12-01
1,10001,Alphington,-37.778316,145.030594,Dec 1 2013 10:00AM,1HR_AV,NO2,Nitrogen Dioxide,3.0,ppb,NO2,2013-12-01
2,10001,Alphington,-37.778316,145.030594,Dec 1 2013 10:00AM,1HR_AV,O3,Ozone,22.0,ppb,O3,2013-12-01
3,10001,Alphington,-37.778316,145.030594,Dec 1 2013 10:00AM,1HR_AV,PM10,TEOM Particles <10micron,13.0,ug/m3,PM10,2013-12-01
4,10001,Alphington,-37.778316,145.030594,Dec 1 2013 10:00AM,1HR_AV,SO2,Sulfur Dioxide,0.0,ppb,SO2,2013-12-01
...,...,...,...,...,...,...,...,...,...,...,...,...
62833,10011,Traralgon,-38.194283,146.531464,2017-12-31 10:00:00,1HR_AV,CO,Carbon Monoxide,0.1,ppm,CO,2017-12-31
62834,10011,Traralgon,-38.194283,146.531464,2017-12-31 10:00:00,1HR_AV,NO2,Nitrogen Dioxide,6.0,ppb,NO2,2017-12-31
62835,10011,Traralgon,-38.194283,146.531464,2017-12-31 10:00:00,1HR_AV,O3,Ozone,17.0,ppb,O3,2017-12-31
62836,10011,Traralgon,-38.194283,146.531464,2017-12-31 10:00:00,1HR_AV,PM10,TEOM Particles <10micron,21.6,ug/m3,PM10,2017-12-31


In [10]:
# 전체 NaN check
aus_1317_api.isna().sum()

sample_point_id              0
sp_name                      0
latitude                     0
longitude                    0
sample_datetime              0
time_basis_id                0
param_id                     0
param_name                   0
value                        0
param_std_unit_of_measure    0
param_short_name             0
date                         0
dtype: int64

In [11]:
# 측정소별 value NaN check
# Alphington 결정

sp_name_list = aus_1317_api['sp_name'].unique().tolist()

for name in sp_name_list: 
    tmp = aus_1317_api.loc[aus_1317_api['sp_name'] == name]
    print(name)
    print('Whole data length: ', len(tmp))
    print(tmp.isna().sum())
    print()

Alphington
Whole data length:  8223
sample_point_id              0
sp_name                      0
latitude                     0
longitude                    0
sample_datetime              0
time_basis_id                0
param_id                     0
param_name                   0
value                        0
param_std_unit_of_measure    0
param_short_name             0
date                         0
dtype: int64

Footscray
Whole data length:  6882
sample_point_id              0
sp_name                      0
latitude                     0
longitude                    0
sample_datetime              0
time_basis_id                0
param_id                     0
param_name                   0
value                        0
param_std_unit_of_measure    0
param_short_name             0
date                         0
dtype: int64

Point Cook
Whole data length:  1572
sample_point_id              0
sp_name                      0
latitude                     0
longitude                   

In [12]:
sp_name_list

['Alphington',
 'Footscray',
 'Point Cook',
 'Altona North',
 'Brighton',
 'Traralgon',
 'Dandenong',
 'Box.Hill',
 'Geelong South',
 'Richmond',
 'Mooroolbark',
 'Melton',
 'Brooklyn',
 'Morwell East',
 'Morwell South',
 'Churchill',
 'Moe',
 'Melbourne CBD']

In [148]:
Alphington_1317_air = aus_1317_api.loc[aus_1317_api['sp_name'] == 'Alphington'].reset_index(drop = True)
Alphington_1317_air

,sample_point_id,sp_name,latitude,longitude,sample_datetime,time_basis_id,param_id,param_name,value,param_std_unit_of_measure,param_short_name,date
0,10001,Alphington,-37.778316,145.030594,Dec 1 2013 10:00AM,1HR_AV,CO,Carbon Monoxide,0.1,ppm,CO,2013-12-01
1,10001,Alphington,-37.778316,145.030594,Dec 1 2013 10:00AM,1HR_AV,NO2,Nitrogen Dioxide,3.0,ppb,NO2,2013-12-01
2,10001,Alphington,-37.778316,145.030594,Dec 1 2013 10:00AM,1HR_AV,O3,Ozone,22.0,ppb,O3,2013-12-01
3,10001,Alphington,-37.778316,145.030594,Dec 1 2013 10:00AM,1HR_AV,PM10,TEOM Particles <10micron,13.0,ug/m3,PM10,2013-12-01
4,10001,Alphington,-37.778316,145.030594,Dec 1 2013 10:00AM,1HR_AV,SO2,Sulfur Dioxide,0.0,ppb,SO2,2013-12-01
...,...,...,...,...,...,...,...,...,...,...,...,...
8218,10001,Alphington,-37.778408,145.030594,2017-12-31 10:00:00,1HR_AV,CO,Carbon Monoxide,0.1,ppm,CO,2017-12-31
8219,10001,Alphington,-37.778408,145.030594,2017-12-31 10:00:00,1HR_AV,NO2,Nitrogen Dioxide,2.0,ppb,NO2,2017-12-31
8220,10001,Alphington,-37.778408,145.030594,2017-12-31 10:00:00,1HR_AV,O3,Ozone,22.0,ppb,O3,2017-12-31
8221,10001,Alphington,-37.778408,145.030594,2017-12-31 10:00:00,1HR_AV,PM10,TEOM Particles <10micron,15.6,ug/m3,PM10,2017-12-31


In [150]:
# long to wide format

Alphington_1317_air_wide = pd.DataFrame(Alphington_1317_air.pivot(index = 'date', columns = 'param_short_name', values = 'value'))
Alphington_1317_air_wide

param_short_name,CO,NO2,O3,PM10,PM2.5,SO2
date,,,,,,
2013-12-01,0.1,3.0,22.0,13.0,NaN,0.0
2013-12-02,0.1,4.0,32.0,27.1,NaN,0.0
2013-12-03,0.3,8.0,17.0,39.3,NaN,2.0
2013-12-04,0.2,12.0,8.0,8.8,NaN,1.0
2013-12-05,0.1,4.0,17.0,11.0,NaN,0.0
...,...,...,...,...,...,...
2017-12-27,0.2,3.0,26.0,12.8,-0.1,0.0
2017-12-28,0.2,12.0,27.0,12.3,6.7,1.0
2017-12-29,0.2,8.0,9.0,27.6,6.4,1.0


In [151]:
for cols in Alphington_1317_air_wide.columns: 
    print(cols, Alphington_1317_air_wide[cols].isna().sum())

CO 69
NO2 79
O3 106
PM10 35
PM2.5 187
SO2 91


In [156]:
# 거리에 따라 쓸 수 있는 측정소만 분리

sp_name_close = ['Alphington', 'Footscray', 'Point Cook', 'Altona North', 'Brighton', 'Box.Hill', 'Richmond', 'Melton', 'Brooklyn', 'Churchill', 
                'Moe', 'Melbourne CBD']

In [182]:
# 가까운 측정소들만 뽑아서 날짜별로 mean 내기
# sp_name_close 말고 그냥 전체 측정소로 하면? 

aus_1317_close_air = aus_1317_api.loc[aus_1317_api['sp_name'].isin(sp_name_list) == True].reset_index(drop = True)
aus_1317_close_air

,sample_point_id,sp_name,latitude,longitude,sample_datetime,time_basis_id,param_id,param_name,value,param_std_unit_of_measure,param_short_name,date
0,10001,Alphington,-37.778316,145.030594,Dec 1 2013 10:00AM,1HR_AV,CO,Carbon Monoxide,0.1,ppm,CO,2013-12-01
1,10001,Alphington,-37.778316,145.030594,Dec 1 2013 10:00AM,1HR_AV,NO2,Nitrogen Dioxide,3.0,ppb,NO2,2013-12-01
2,10001,Alphington,-37.778316,145.030594,Dec 1 2013 10:00AM,1HR_AV,O3,Ozone,22.0,ppb,O3,2013-12-01
3,10001,Alphington,-37.778316,145.030594,Dec 1 2013 10:00AM,1HR_AV,PM10,TEOM Particles <10micron,13.0,ug/m3,PM10,2013-12-01
4,10001,Alphington,-37.778316,145.030594,Dec 1 2013 10:00AM,1HR_AV,SO2,Sulfur Dioxide,0.0,ppb,SO2,2013-12-01
...,...,...,...,...,...,...,...,...,...,...,...,...
62833,10011,Traralgon,-38.194283,146.531464,2017-12-31 10:00:00,1HR_AV,CO,Carbon Monoxide,0.1,ppm,CO,2017-12-31
62834,10011,Traralgon,-38.194283,146.531464,2017-12-31 10:00:00,1HR_AV,NO2,Nitrogen Dioxide,6.0,ppb,NO2,2017-12-31
62835,10011,Traralgon,-38.194283,146.531464,2017-12-31 10:00:00,1HR_AV,O3,Ozone,17.0,ppb,O3,2017-12-31
62836,10011,Traralgon,-38.194283,146.531464,2017-12-31 10:00:00,1HR_AV,PM10,TEOM Particles <10micron,21.6,ug/m3,PM10,2017-12-31


In [183]:
# 날짜별 groupby and mean

aus_1317_close_air_groupby = pd.DataFrame(aus_1317_close_air.groupby(['date', 'param_short_name']).mean(['value'])).reset_index()
aus_1317_close_air_groupby

,date,param_short_name,sample_point_id,latitude,longitude,value
0,2013-12-01,CO,10067.666667,-37.863920,144.956421,0.100000
1,2013-12-01,NO2,10043.500000,-37.906200,145.088751,3.600000
2,2013-12-01,O3,10048.700000,-37.894518,145.045038,21.600000
3,2013-12-01,PM10,10065.900000,-37.909900,145.131380,19.650000
4,2013-12-01,SO2,10028.333333,-37.937335,145.133385,1.666667
...,...,...,...,...,...,...
8881,2017-12-31,NO2,10067.800000,-38.038084,145.438492,2.600000
8882,2017-12-31,O3,10081.125000,-37.949770,145.230570,21.250000
8883,2017-12-31,PM10,10078.500000,-37.924699,145.163462,16.066667
8884,2017-12-31,PM2.5,10115.625000,-38.068054,145.861235,4.075000


In [190]:
for cols in aus_1317_close_air_groupby.columns: 
    print(cols, aus_1317_close_air_groupby[cols].isna().sum())

date 0
param_short_name 0
sample_point_id 0
latitude 0
longitude 0
value 0


In [184]:
# long to wide format

aus_1317_close_air_wide = pd.DataFrame(aus_1317_close_air_groupby.pivot(index = 'date', columns = 'param_short_name', values = 'value')).reset_index()
aus_1317_close_air_wide

param_short_name,date,CO,NO2,O3,PM10,PM2.5,SO2
0,2013-12-01,0.100000,3.600000,21.600000,19.650000,NaN,1.666667
1,2013-12-02,0.100000,5.000000,29.800000,36.770000,NaN,0.800000
2,2013-12-03,0.166667,6.111111,21.100000,34.250000,NaN,3.000000
3,2013-12-04,0.150000,8.000000,11.200000,8.340000,NaN,0.833333
4,2013-12-05,0.050000,3.444444,16.200000,14.380000,NaN,1.500000
...,...,...,...,...,...,...,...
1487,2017-12-27,0.133333,2.833333,28.111111,21.500000,5.3375,0.500000
1488,2017-12-28,0.150000,10.200000,26.250000,26.433334,7.9125,1.400000
1489,2017-12-29,0.120000,3.750000,16.428571,23.916667,9.2000,0.833333
1490,2017-12-30,0.066667,2.200000,15.750000,10.233333,5.7250,1.500000


In [185]:
for cols in aus_1317_close_air_wide.columns: 
    print(cols, aus_1317_close_air_wide[cols].isna().sum())

date 0
CO 1
NO2 1
O3 1
PM10 2
PM2.5 60
SO2 1


In [ ]:
aus_1317_close_air_wide.loc[aus_1317_close_air_wide['PM2.5'].isna() == True]

In [187]:
60*25

1500

In [189]:
aus_1317_close_air_wide.to_csv('D:\\SNUlab\\0. data\\thermal inversion\\Australia_Victoria_Air\\aus_1317_air_mean_by_pollutants.csv', encoding = 'euc-kr', index = False)

In [153]:
Alphington_1317_air_wide.to_csv('D:\\SNUlab\\0. data\\thermal inversion\\Australia_Victoria_Air\\alphington_1317_air_by_pollutants.csv', encoding = 'euc-kr')